<a href="https://colab.research.google.com/github/csmsum/ml4ai/blob/main/oct6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
import numpy as np
import pandas as pd

weather = pd.read_csv('/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04/data/nyc_weather_2018.csv', index_col='date', parse_dates=True)
weather.head()

,datatype,station,attributes,value
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,",,N,",0.0
2018-01-01,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2018-01-01,SNOW,GHCND:US1NJBG0015,",,N,",0.0
2018-01-01,PRCP,GHCND:US1NJBG0017,",,N,",0.0
2018-01-01,SNOW,GHCND:US1NJBG0017,",,N,",0.0


In [3]:
fb = pd.read_csv('/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04/data/fb_2018.csv', index_col='date', parse_dates=True)
fb.head()

,open,high,low,close,volume
date,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903
2018-01-03,181.88,184.78,181.3300,184.67,16886563
2018-01-04,184.90,186.21,184.0996,184.33,13880896
2018-01-05,185.59,186.90,184.9300,186.85,13574535
2018-01-08,187.20,188.90,186.3300,188.28,17994726


표준화

In [4]:
fb.assign(
    abs_z_score_volume=lambda x: \
        x.volume.sub(x.volume.mean()).div(x.volume.std()).abs()
).query('abs_z_score_volume > 3')

,open,high,low,close,volume,abs_z_score_volume
date,,,,,,
2018-03-19,177.01,177.17,170.06,172.56,88140060,3.145078
2018-03-20,167.47,170.20,161.95,168.15,129851768,5.315169
2018-03-21,164.80,173.40,163.30,169.39,106598834,4.105413
2018-03-26,160.82,161.10,149.02,160.06,126116634,5.120845
2018-07-26,174.89,180.13,173.75,176.26,169803668,7.393705


pct_change에 랭크 매겨서 

In [8]:
fb.assign(
    volume_pct_change=fb.volume.pct_change(),
    pct_change_rank=lambda x: \
        x.volume_pct_change.abs().rank(ascending=False)
).nsmallest(5, 'pct_change_rank')

,open,high,low,close,volume,volume_pct_change,pct_change_rank
date,,,,,,,
2018-01-12,178.06,181.48,177.40,179.37,77551299,7.087876,1.0
2018-03-19,177.01,177.17,170.06,172.56,88140060,2.611789,2.0
2018-07-26,174.89,180.13,173.75,176.26,169803668,1.628841,3.0
2018-09-21,166.64,167.25,162.81,162.93,45994800,1.428956,4.0
2018-03-26,160.82,161.10,149.02,160.06,126116634,1.352496,5.0


Qcut을 이용하면 quantile에 맞게 비닝할 수 있다.

In [16]:
volume_qbinned = pd.qcut(fb.volume, q=4, labels=['q1', 'q2', 'q3', 'q4'])
volume_qbinned.value_counts()

q1    63
q2    63
q4    63
q3    62
Name: volume, dtype: int64

Apply를 통해 함수를 적용시킬 수 있다. 먼저 데이터 한번 보자

In [19]:
central_park_weather = weather\
    .query('station == "GHCND:USW00094728"')\
    .pivot(index='date', columns='datatype', values='value')

In [23]:
oct_weather_z_score = central_park_weather\
    .loc['2018-10', ['TMIN', 'TMAX', 'PRCP']]\
    .apply(lambda x: x.sub(x.mean()).div(x.std()))
oct_weather_z_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
datatype,,,,,,,,
TMIN,31.0,-1.790682e-16,1.0,-1.339112,-0.751019,-0.474269,1.065152,1.843511
TMAX,31.0,1.951844e-16,1.0,-1.305582,-0.870013,-0.138258,1.011643,1.604016
PRCP,31.0,4.655774e-17,1.0,-0.394438,-0.394438,-0.394438,-0.240253,3.936167


In [28]:
fb.apply(
    lambda x: np.vectorize(lambda y: len(str(np.ceil(y))))(x)
).astype('int64').equals(
    fb.applymap(lambda x: len(str(np.ceil(x))))
)

True

##1.1Find which quarter had the most rain from the daily precipitation data. 

In [44]:
weather

,date,datatype,station,attributes,value
0,2018-01-01,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01,SNOW,GHCND:US1NJBG0017,",,N,",0.0
...,...,...,...,...,...
78775,2018-12-31,WDF5,GHCND:USW00094789,",,W,",130.0
78776,2018-12-31,WSF2,GHCND:USW00094789,",,W,",9.8
78777,2018-12-31,WSF5,GHCND:USW00094789,",,W,",12.5
78778,2018-12-31,WT01,GHCND:USW00094789,",,W,",1.0


In [48]:
weather.query('datatype == "PRCP"')\
    .groupby(level=0).mean()\
    .groupby(pd.Grouper(freq='Q')).sum().value.nlargest(5)

date
2018-09-30    547.665911
2018-12-31    474.536973
2018-03-31    364.840077
2018-06-30    334.979325
Name: value, dtype: float64

##1.2 Assign ‘quarterly total precipitation’ and ‘number of days in a month’ variables to daily precipitation

In [91]:
weather.query('datatype == "PRCP"')\
    .rename(dict(value='prcp'), axis = 1).groupby(level=0).mean().assign(\
    quarterly_total_prcp = lambda x: x.groupby(pd.Grouper(freq='Q')).transform(np.sum),\
    num_o_days = lambda x: x.groupby(pd.Grouper(freq='Q')).value)

AttributeError: ignored

In [60]:
np.sum

<function numpy.sum(a, axis=None, dtype=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>

#Task 3

In [85]:
fb.pivot_table(index = 'trading_volume', columns = fb.index.month, values = 'close', aggfunc = 'count')

date,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0
